In [3]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import time
import re
import pandas as pd
from urllib.request import urlopen
from urllib.parse import quote_plus

In [38]:
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url

def instagram_login():

    # 개인정보 보호 오류 창... 
    driver.find_element_by_css_selector('#details-button').click(); time.sleep(1)
    driver.find_element_by_css_selector('#proceed-link').click(); time.sleep(2)
    # 로그인
    driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys('아이디')
    driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys('비밀번호')
    driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div').click()
    time.sleep(3)
    # 로그인 정보 저장 (나중에 하기)
    driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button').click()
    time.sleep(3) 
    # 알림설정 (나중에 하기)
    driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/button[2]').click()
    time.sleep(3)

# 첫게시물 보기
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)

# 다음게시글
def move_next(driver):
    right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

# 내용 훔쳐오기
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
        
    # 해시태그 > 정규표현식
    tags = re.findall(r'#[^\s#,\\]+', content) 
    # 작성일자
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # 좋아요 수
    try:
        like = soup.select('a.zV_Nj')[0].text[4:-1]   
    except:
        like = 0
    # 위치정보
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
    # 리스트로 저장
    data = [content, date, like, place, tags]
    return data

In [46]:
driver = webdriver.Chrome("chromedriver.exe")
driver.get('https://www.instargram.com')

instagram_login() 

# 검색결과
driver.get(insta_searching('일기'))
time.sleep(4)

# 검색결과물 개수만큼 크롤링
target_temp = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/header/div[2]/div/div[2]/span/span').text
target = int(target_temp.replace(',', ''))
time.sleep(3)

select_first(driver)
results = [ ]

for i in range(target):
    # 수집 오류 > 2초 대기
    try:
        data = get_content(driver)    # 게시글 정보
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)

driver.close()

In [ ]:
df = pd.DataFrame(results)
df.columns = ['content','data','like','place','tags']

#df.to_excel('일기 해시태그.xlsx', index=False) 

---
# 스프레드시트 연동
---

In [61]:
pip install --upgrade oauth2client

Note: you may need to restart the kernel to use updated packages.


In [62]:
pip install gspread

Note: you may need to restart the kernel to use updated packages.


In [63]:
pip install PyOpenSSL

Note: you may need to restart the kernel to use updated packages.


In [69]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive',
]


json_file_name = '파일명.json'

credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

spreadsheet_url = '시트 url'

# 스프레스시트 문서 가져오기 
doc = gc.open_by_url(spreadsheet_url)

# 시트 선택하기
worksheet = doc.worksheet('일기')

## 데이터 가져오기

In [70]:
# 특정 셀 가져오기
cell_data = worksheet.acell('A1').value
print(cell_data)

content


In [71]:
# 행 데이터 가져오기
row_data = worksheet.row_values(1)
print(row_data)

['content', 'data', 'like', 'place', 'tags']


In [72]:
# 열 데이터 가져오기
column_data = worksheet.col_values(1)
print(column_data)

['content']


In [73]:
# 특정 영역 선택해 가져오기
range_list = worksheet.range('A1:C1')
print(range_list)

for cell in range_list:
    print(cell.value)

[<Cell R1C1 'content'>, <Cell R1C2 'data'>, <Cell R1C3 'like'>]
content
data
like


## 데이터 쓰기

In [74]:
# 특정 셀에 데이터 쓰기
worksheet.update_acell('A2', 'a2 update')

{'spreadsheetId': '153tefpUd5JMxR7iWw3HtPW1zzsdruo9yAQgVxi9Boo0',
 'updatedRange': "'일기'!A2",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [75]:
# 행 데이터 쓰기
# worksheet.append_row(['new1', 'new2']) >> 맨 아래줄에 추가
worksheet.insert_row(['a', 'b', 'c', 'd', 'e'], 2)

{'spreadsheetId': '153tefpUd5JMxR7iWw3HtPW1zzsdruo9yAQgVxi9Boo0',
 'updates': {'spreadsheetId': '153tefpUd5JMxR7iWw3HtPW1zzsdruo9yAQgVxi9Boo0',
  'updatedRange': "'일기'!A2:E2",
  'updatedRows': 1,
  'updatedColumns': 5,
  'updatedCells': 5}}

## 시트 크기 조정

In [106]:
# worksheet.resize(5 ,5)

## 크롤링해서 데이터 추가

In [109]:
driver = webdriver.Chrome("chromedriver.exe")
driver.get('https://www.instargram.com')

instagram_login() 

driver.get(insta_searching('일기')) # 검색어 일기
time.sleep(4)

target_temp = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/header/div[2]/div/div[2]/span/span').text
target = int(target_temp.replace(',', ''))
time.sleep(3)

select_first(driver)

for i in range(target):
    # 수집 오류 > 2초 대기
    try:
        data = get_content(driver)    # 게시글 정보
        worksheet.insert_row([data[0], data[1], data[2], data[3], str(data[4])], 2)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)

driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=92.0.4515.159)
